<a href="https://colab.research.google.com/github/SablArt/LR2nd/blob/main/%D0%AD%D0%BB%D0%B8%D0%BD%D1%842.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 1. Подключение библиотек
import pandas as pd     # для чтения и записи табличных файлов
import numpy as np      # для математической обработки данных
import matplotlib.pyplot as plt # для построения графиков

din = {}

# Получение и чтение Excel файла из облака

url = "https://drive.google.com/uc?export=download&id=130uKatSQ7Tpm4cAED2Q5H5C8WQSdDm8R"
input_df = pd.read_excel(url, decimal=',') 

# Берем только временной ряд скорости ветра в м/с
# iloc - выделяет из таблицы указанный фрагмент, в данном случае все строки и только первый столбец
wind_speed = input_df.iloc[: , 0].values

# Задаём зависимость среднечасовой мощности, вырабатываемой ВЭУ, от средней скорости ветра
def wind_speed_to_power(speed):
  min_speed = 5
  max_speed = 20
  critical_speed = 25
  
  # выработка равна нулю, если скорость ветра выше критической или ниже минимальной
  # поэтому обнуляем такую скорость
  speed_tmp = np.where(np.logical_or(speed > critical_speed, speed < min_speed), 0, speed)

  # если скорость выше максимальной, то выработка не будет расти, добавляем ограничение сверху 
  speed_tmp = np.clip(speed_tmp, a_min = None, a_max = max_speed)

  # на рабочем участке от min_speed до max_speed мощность пропорциональна кубу скорости
  power = 0.0017 * (speed_tmp) ** 3
  return power

# Строим интерактивный график зависимости мощности от скорости ветка при помощи библиотеки Bokeh
from bokeh.plotting import figure, output_file, show
from bokeh.io import output_notebook

#ПОСУТОЧНЫЙ
days = len(wind_speed)//24
wind_speed_days = []
for i in range(1, days):
    din["X" + str(i)] = wind_speed[24*(i-1):i*24]
    din["Graph" + str(i)] = sum(din["X" + str(i)])/24
    wind_speed_days.append(din["Graph" + str(i)])
#КОНЕЦ ПОСУТОЧНОГО

output_notebook()

p1 = figure(plot_width = 800, plot_height = 400, title="Почасовой график выработки мощности")
p1.line(np.arange(len(wind_speed)), wind_speed_to_power(wind_speed), line_width = 2,
       legend_label="Мощность")
p1.line(np.arange(len(wind_speed)), wind_speed, line_width = 2, color = 'green',
       legend_label="Скорость ветра")
show(p1)

#ПОСУТОЧНЫЙ
p2 = figure(plot_width = 800, plot_height = 400, title="График среднесуточной выработки мощности")
p2.line(np.arange(len(wind_speed_days)), wind_speed_to_power(wind_speed), line_width = 2,
       legend_label="Мощность")
p2.line(np.arange(len(wind_speed_days)), wind_speed_days, line_width = 2, color = 'green',
       legend_label="Скорость ветра")
show(p2)